# Parte 4: Clasificación con Capsnet

La implementación de esta red Capsnet se ha basado en el código implementado por Aurélien Géron (https://github.com/ageron/handson-ml/blob/master/extra_capsnets.ipynb) 

### Resultados con la arquitectura anterior: 



Resultados:
    
    TRAIN                   DEV
    loss       accuracy     val_loss    val_accuracy
       0.870418	0.807692	0.951864	0.730769
    
Por tanto: 

    E = 1 - Accuracy
    Etrain = 1 - 0.807692 = 0.192308
    Etest = 1 - 0.730769 = 0.269231
    
    Bias = Etrain - Ehuman = 0.192308
    Variance = Etest - Etrain = 0.269231 - 0.192308 = 0.076923

La varianza se ha reducido muchisimo pero el bias ha aumentado a un 19%. Se tratará de mejorar el bias. Para mejorar esto será necesario añadir más complejidad, elegir una mejor optimización, cambiando la arquitectura (más neuronas, más capas)... 


### Cambios realizados

Se aumenta de 100 a 500

### Nuevos resultados: 

### 1 - Import Libraries

In [1]:
# Tensorflow and tf.keras
import tensorflow as tf
from tensorflow import keras

#Helper libraries
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

# Signal libraries
from scipy import signal

In [2]:
# Reset the default graph, in case you re-run this notebook without restarting the kernel:
tf.compat.v1.reset_default_graph()

# Random seeds so that this notebook always produces the same output:
np.random.seed(42)
tf.random.set_seed(45)

### 2 - Load data

In [3]:
class ROutput:
    def __init__(self, task, data):
        self.task = task
        self.data = data
        
class OutTaskData: 
    def __init__(self, task, data): 
        self.task = task
        self.data = data

In [4]:
import scipy.io as sio
# Primero leemos los registros
def read_outputs(rec):
    '''read_outputs("userS0091f1.mat")'''
    mat = sio.loadmat(rec)
    mdata = mat['session']
    val = mdata[0,0]
    #output = ROutput(np.array(val["task"]), np.array(val["data"]))
    output = ROutput(np.array(val["task_EEG_p"]), np.array(val["data_processed_EEG"]))
    return output

### Cargamos los datos

In [5]:
# Configuración
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.linear_model import Perceptron
from tensorflow.keras.utils import to_categorical


task1 = 402 # SE PUEDE CAMBIAR
task2 = 404 # SE PUEDE CAMBIAR
task_OneHotEnconding = {402: [1.,0.], 404: [0.,1.]}
user = 'W29' # SE PUEDE CAMBIAR
day = '0329'
folder_day = 'W29-29_03_2021'
total_records = 22 # CAMBIAR SI HAY MAS REGISTROS
fm = 200
electrodes_names_selected = ['F3', 'FZ', 'FC1','FCZ','C1','CZ','CP1','CPZ', 'FC5', 'FC3','C5','C3','CP5','CP3','P3',
                             'PZ','F4','FC2','FC4','FC6','C2','C4','CP2','CP4','C6','CP6','P4','HR' ,'HL', 'VU', 'VD']
number_channels = len(electrodes_names_selected)

In [6]:
lTaskData = []
total_records_used = 0
for i_rec in range(1,total_records+1):
    i_rec_record = i_rec
    if i_rec_record <10:
        i_rec_record = "0"+str(i_rec_record)
    if i_rec % 2 == 0: # Registros impares primero: USUARIO SIN MOVIMIENTO SOLO PENSANDO
        record = "./RegistrosProcesados2/"+folder_day+"/W29_2021"+day+"_openloop_"+str(i_rec_record)+"_processed.mat"
        output = read_outputs(record) # output.task será y, output.data será x


        output.task = np.transpose(output.task)
        output.data = output.data.reshape((np.shape(output.data)[0],np.shape(output.data)[1]))
        output.data = np.transpose(output.data)
        #output.data = output.data.reshape((np.shape(output.data)[0],np.shape(output.data)[1],1))

        outT = (output.task == task1) | (output.task == task2)
        outData = output.data[0:np.shape(output.data)[0], outT[0,:]]
        outTask = output.task[0, outT[0,:]]
        outTD = OutTaskData(outTask, outData)

        lTaskData.append(outTD)
        total_records_used+=1
print(total_records_used, total_records)

11 22


In [7]:
# Vamos a coger 2 registros para el entrenamiento, 1 para el conjunto dev set, 1 para el test set
X_train, y_train, X_dev, y_dev, X_test, y_test = [],[],[],[],[],[] 
for j in range(0,total_records_used-3): # Cogemos 18 registros para entrenamiento
    X_train.append(lTaskData[j].data)
    y_train.append(lTaskData[j].task)

for j in range(total_records_used-3,total_records_used-1): # Cogemos 2 registros para el dev set
    X_dev.append(lTaskData[j].data)
    y_dev.append(lTaskData[j].task)
for j in range(total_records_used-1,total_records_used): # Cogemos 2 registros para el test set
    X_test.append(lTaskData[j].data)
    y_test.append(lTaskData[j].task)

X_train = np.array(X_train)
#y_train = np.ravel(np.array(y_train))
y_train = np.array(y_train)
X_dev = np.array(X_dev)
#y_dev = np.ravel(np.array(y_dev))
y_dev = np.array(y_dev)
X_test = np.array(X_test)
y_test = np.array(y_test)
#y_test = np.ravel(np.array(y_test))

print ("X_train:",X_train.shape)
print ("y_train:",y_train.shape)
print ("X_dev:",X_dev.shape)
print ("y_dev:",y_dev.shape)
print ("X_test:",X_test.shape)
print ("y_test:",y_test.shape)





# VENTANEO Y ONE HOT ENCODING 
window = 5
samples_advance = 3

# Ventaneo X_train

X_train_l = []
y_train_l = []
for num_X_train in range(np.shape(X_train)[0]): # Para no mezclar registros
    win_init = int(0)
    window_position = 0
    
    for i in range(np.shape(X_train)[2]): # For each signal registered
        win_end = int(win_init + window)
        if win_end >= np.shape(X_train)[2]:
            break

        task = np.unique(y_train[num_X_train,win_init:win_end])

        if len(task)==1:
        #if task1 in task or task2 in task:
            signal_window = X_train[num_X_train, :, win_init:win_end]
            
            #data_filtered = preprocessing(signal_window, fm, number_channels)
            #X_train_l.append(data_filtered)
            X_train_l.append(signal_window)
            #taskOH = task_OneHotEnconding[task[0]]
            #y_train_l.append(taskOH)
            y_train_l.append(task)
            
            
        win_init += int(samples_advance)

X_train_l = np.array(X_train_l)
y_train_l = np.array(y_train_l)


# Ventaneo X_dev
X_dev_l = []
y_dev_l = []
for num_X_dev in range(np.shape(X_dev)[0]):
    win_init = int(0)
    window_position = 0
    
    for i in range(np.shape(X_dev)[2]): # For each signal registered
        win_end = int(win_init + window)
        if win_end >= np.shape(X_dev)[2]:
            break

        task = np.unique(y_dev[num_X_dev,win_init:win_end])

        if len(task)==1:
        #if task1 in task or task2 in task:
            signal_window = X_dev[num_X_dev, :, win_init:win_end]
            
            #data_filtered = preprocessing(signal_window, fm, number_channels)
            #X_train_l.append(data_filtered)
            X_dev_l.append(signal_window)
            #taskOH = task_OneHotEnconding[task[0]]
            #y_dev_l.append(taskOH)
            y_dev_l.append(task)
            
        win_init += int(samples_advance)

X_dev_l = np.array(X_dev_l)
y_dev_l = np.array(y_dev_l)

# Ventaneo X_test
X_test_l = []
y_test_l = []
for num_X_test in range(np.shape(X_test)[0]): 
    win_init = int(0)
    window_position = 0
    
    for i in range(np.shape(X_test)[2]): # For each signal registered
        win_end = int(win_init + window)
        if win_end >= np.shape(X_test)[2]:
            break

        task = np.unique(y_test[num_X_test,win_init:win_end])

        if len(task)==1:
        #if task1 in task or task2 in task:
            signal_window = X_test[num_X_test, :, win_init:win_end]
            
            #data_filtered = preprocessing(signal_window, fm, number_channels)
            #X_train_l.append(data_filtered)
            X_test_l.append(signal_window)
            #taskOH = task_OneHotEnconding[task[0]]
            y_test_l.append(task)
            #y_test_l.append(taskOH)
            
        win_init += int(samples_advance)

X_test_l = np.array(X_test_l)
y_test_l = np.array(y_test_l)




X_train_l = X_train_l.reshape((np.shape(X_train_l)[0],np.shape(X_train_l)[1],np.shape(X_train_l)[2], 1))
X_dev_l = X_dev_l.reshape((np.shape(X_dev_l)[0],np.shape(X_dev_l)[1],np.shape(X_dev_l)[2], 1))
X_test_l = X_test_l.reshape((np.shape(X_test_l)[0],np.shape(X_test_l)[1],np.shape(X_test_l)[2], 1))

print()
print("ONE HOT ENCODER & WINDOWING:")
print ("X_train:",X_train_l.shape)
print ("y_train:",y_train_l.shape)
print ("X_dev:",X_dev_l.shape)
print ("y_dev:",y_dev_l.shape)
print ("X_test:",X_test_l.shape)
print ("y_test:",y_test_l.shape)

X_train = X_train_l
y_train = y_train_l.reshape((np.shape(y_train_l)[0]))
X_dev = X_dev_l
y_dev = y_dev_l.reshape((np.shape(y_dev_l)[0]))
X_test = X_test_l
y_test = y_test_l.reshape((np.shape(y_test_l)[0]))

X_train = X_train.astype('float32')
X_dev = X_dev.astype('float32')
X_test = X_test.astype('float32')
y_train = y_train.astype('int64')
y_dev = y_dev.astype('int64')
y_test = y_test.astype('int64')

X_train: (8, 32, 49)
y_train: (8, 49)
X_dev: (2, 32, 49)
y_dev: (2, 49)
X_test: (1, 32, 49)
y_test: (1, 49)

ONE HOT ENCODER & WINDOWING:
X_train: (104, 32, 5, 1)
y_train: (104, 1)
X_dev: (26, 32, 5, 1)
y_dev: (26, 1)
X_test: (13, 32, 5, 1)
y_test: (13, 1)


### 4 - Construcción de la Capsnet 

In [8]:
from keras.layers.convolutional import Conv2D

#### Datos de entrada: 
Se comienza creando un placeholder para los datos de entrada. Un placeholder es una variable a la que se le asigna datos después. Se utiliza para alimentar ejemplos de entrenamiento reales. 

In [9]:
tf.compat.v1.disable_eager_execution()
X = tf.compat.v1.placeholder(shape=[None, 32, 5, 1], dtype=tf.float32, name="X")

#### Capsulas primarias 

La primera capa estará compuesta de 128 mapas de características con 5x5 capsulas cada una, donde cada capsula tendrá como salida un vector de activación 4D

In [10]:
caps1_n_maps = 128 
caps1_n_caps = caps1_n_maps * 5 * 5 # 3200 capsulas primarias 
caps1_n_dims = 4

Para calcular sus salidas, primero se aplican dos capas convolucionales regulares: 

In [11]:
conv1_params = { # PRIMERA CAPA CONVOLUCIONAL
    "filters": 4, # 4,
    "kernel_size": 3,
    "strides": 1,
    "padding": "same",
    "activation": tf.nn.relu # Se utiliza una función ReLu en vez de la SeLu utilizada en el artículo de Capsnet - EEG
}
conv2_params = { # CÁPSULAS PRIMARIAS
    "filters": caps1_n_maps * caps1_n_dims, # 512 filtros creados porque hay 128 cápsulas con una dimensión de 4 lo que hace un total de 512 filtros
    "kernel_size": 3,
    "strides": 2,
    "padding": "same",
    "activation": tf.nn.relu # Se utiliza una función ReLu en vez de la SeLu utilizada en el artículo de Capsnet - EEG
}

In [12]:
conv1 = tf.keras.layers.Conv2D(name="conv1", **conv1_params, input_shape=(32, 5, 1 ))(X)
print(conv1.shape) # (32-3+0)/1 + 1 = 30; (5-3+0)/1 +1 = 3; 4 filtros => (30,3,4)
conv2 = tf.keras.layers.Conv2D( name="conv2", **conv2_params)(conv1)
print(conv2.shape) # (30-3+0)/2 + 1 = 14; (3-3+0)/2 +1 = 1; 4 filtros => (30,3,512)

(None, 32, 5, 4)
(None, 16, 3, 512)


A continuación, cambiamos la forma de la salida para obtener un grupo de vectores 8D que representan las salidas de las cápsulas primarias.

La salida de la conv2 es un array que contiene 3200 (128x4) mapas de características para cada instancia, donde cada mapa de características es 5x5. Por tanto, el tamaño de la salida será (tamaño del batch,5,5,3200). Se busca dividir los 3200 en 128 vectores de 4 dimensiones cada uno. Eso se puede lograr haciendo reshape (cambiar la forma) a (tamaño del batch,5,5,128,4). Sin embargo, como esta primera capa de cápsula está completamente conectada a la siguiente capa de cápsula, se puede simplemente aplanar (flatten) con rejillas (grids) de 6x6. Esto significa con que sirve con hacer reshape a (tamaño del batch, 5x5x128,4)

In [13]:
caps1_raw = tf.reshape(conv2, [-1, caps1_n_caps, caps1_n_dims],
                       name="caps1_raw")
caps1_raw

<tf.Tensor 'caps1_raw:0' shape=(None, 3200, 4) dtype=float32>

Ahora, es necesario utilizar la función squash en todos estos vectores para normalizarlos. 

Es notorio que no se puede usar tf.norm() porque la derivada de ||s|| no está definida cuando ||s||=0, ya que si un vector es 0 explotará durante el entrenamiento (explosión de gradiente, lo hemos visto en Deep), es decir, los gradientes serán nan, por lo que cuando el optimizador actualice las variables, también se convertirán en nan, y de ahí en adelante quedará atrapado en nan. La solución es implementar la norma manualmente calculando la raíz cuadrada de la suma de cuadrados más un pequeño valor épsilon: $\left \| s \right \| \approx \sqrt{\sum_{i}s_{i}^{2}+\epsilon }$

In [14]:
def squash(s, axis=-1, epsilon=1e-7, name=None):
    # Función actualizada extraída de Aurélien Géron
    with tf.name_scope(name):
        squared_norm = tf.reduce_sum(tf.square(s), axis=axis, keepdims=True)
        safe_norm = tf.sqrt(squared_norm + epsilon)
        squash_factor = squared_norm / (1. + squared_norm)
        unit_vector = s / safe_norm
        return squash_factor * unit_vector

Ahora se aplica esta función para obtener la salida $u_{i}$ para cada cápsula primaria i:

In [15]:
caps1_output = squash(caps1_raw, name="caps1_output")
np.shape(caps1_output)

TensorShape([None, 3200, 4])

Así ya se tiene calculada la salida de la primera capa de cápsula. La dificultad comienza ahora ya que habrá que utilizar el algoritmo de enrutamiento dinámico entre las cápsulas primarias y las cápsulas MI.

#### Cápsulas MI
Para calcular la salida de las cápsulas MI, primero se deben calcular los vectores de salida predichos (uno para cada par de cápsulas primaria/MI). Entonces, se podrá correr el algoritmo de enrutamiento dinámico por acuerdos.  

###### Calculando los vectores de salida predichos
La capa de cápsula MI contendrá 2 cápsulas (uno por cada tarea) de 8 dimensiones cada una: 

In [16]:
caps2_n_caps = 2
caps2_n_dims = 8

Para cada cápsula i en la primera capa, se busca predecir la salida de cadapa cápsula j en la segunda capa. Para ello, se necesitará una matriz de transformación $W_{ij}$ (una por cada par de cápsulas (i,j)), entonces se podrá calcular la salida predicha $\hat{u}_{j|i} = W_{ij}u_{i}$. Como se quiere transformar un vector 4D en un vector 8D, cada matriz de transformación $W_{ij}$ debe tener una dimensión (shape) de (8,4),

Para calcular $\hat{u}_{j|i}$ para cada par de cápsulas (i,j), se usará la función tf.matmul() que multiplica arrays de grandes dimensiones. 

El primer array tiene unas dimensiones de (tamaño del batch, 3200 capsulas primarias, 2, 8, 4) y la forma del segundo array es (tamaño del batch, 3200, 2, 4, 1).

Las cápsulas de la primera capa en realidad ya generan predicciones para los datos de tamaño batch, por lo que la segunda matriz estará bien, pero para la primera matriz, necesitaremos usar tf.tile () para tener x (del tamaño del batch) copias de las matrices de transformación.

Se va a comenzar creanto una variable de entrenamiento W de dimensión (1, 3200 , 2, 8, 4) que contendrá todas las matrices de transformación, la primera dimensión de tamaño 1 hará que sea fácil realizar las copias. Se iniciará esta variable aleatoriamente usando una distribución normal con una desviación estandar 0.1.

In [17]:
init_sigma = 0.1

W_init = tf.random.normal(
    shape=(1, caps1_n_caps, caps2_n_caps, caps2_n_dims, caps1_n_dims),
    stddev=init_sigma, dtype=tf.float32, name="W_init")
W = tf.Variable(W_init, name="W")

Ahora podemos crear la primera matriz repitiendo W una vez por instancia:

In [18]:
batch_size = tf.shape(X)[0]
W_tiled = tf.tile(W, [batch_size, 1, 1, 1, 1], name="W_tiled")

Ahora vamos con la segunda matriz, que como se comentó tiene que ser de dimensión (tamaño del batch, 3200, 2, 4, 1) que contiene la salida de las cápsulas de la primera capa repetidas 2 veces (una vez por tarea, a lo largo de la tercera dimension, con axis=2). La salida de la capa 1 tiene una dimensión (tamaño del batch, 3200, 4) por lo que primero debemos expandirla dos veces, para obtener una matriz de forma (tamaño del batch, 3200, 1, 4, 1), entonces esto se podrá repetir 2 veces a lo largo de la tercera dimensión.

In [19]:
caps1_output_expanded = tf.expand_dims(caps1_output, -1,
                                       name="caps1_output_expanded")
caps1_output_tile = tf.expand_dims(caps1_output_expanded, 2,
                                   name="caps1_output_tile")
caps1_output_tiled = tf.tile(caps1_output_tile, [1, 1, caps2_n_caps, 1, 1],
                             name="caps1_output_tiled")

Se comprueban las dimensiones del primer array:

In [20]:
W_tiled

<tf.Tensor 'W_tiled:0' shape=(None, 3200, 2, 8, 4) dtype=float32>

Ya es posible predecir vectores de salida $\hat{u}_{j|i}$. Para ello se multiplican estos dos arrays usando tf.matmul() como se explico anteriormente:

In [21]:
caps2_predicted = tf.matmul(W_tiled, caps1_output_tiled,
                            name="caps2_predicted")
caps2_predicted # Se comprueba la dimensión

<tf.Tensor 'caps2_predicted:0' shape=(None, 3200, 2, 8, 1) dtype=float32>

Por tanto, para cada instancia en el batch (que aún no se ha instanciado) y para cada par de primeras y segundas capas de cápsulas (3200x2) se tiene un vector de salida de predicción de 8D (8x1). 

Es el momento de aplicar el algoritmo de enrutamiento dinámico por acuerdos: 

#### Routing by agreement

Primero, se inicializan los pesos inicialies $b_{ij}$ a 0.

In [22]:
raw_weights = tf.zeros([batch_size, caps1_n_caps, caps2_n_caps, 1, 1],
                       dtype=np.float32, name="raw_weights")
# Las últimas 2 dimensiones tienen tamaño 1, ahora se explicará el por qué
raw_weights

<tf.Tensor 'raw_weights:0' shape=(None, 3200, 2, 1, 1) dtype=float32>

###### Ronda 1
Se aplica la función softmax para calcular los pesos de routing $c_{i}=softmax(b_{i})$

In [23]:
routing_weights = tf.nn.softmax(raw_weights, name="routing_weights")
routing_weights

<tf.Tensor 'routing_weights:0' shape=(None, 3200, 2, 1, 1) dtype=float32>

Ahora se va a calcular la net para todos las salidas predichas de los vectores para cada capsula de la segunda capa, $s_{j}= \sum_{i} c_{ij}\hat{u}_{j|i}$

In [24]:
weighted_predictions = tf.multiply(routing_weights, caps2_predicted,
                                   name="weighted_predictions")# realiza multiplicación de matrices por elementos

weighted_sum = tf.reduce_sum(weighted_predictions, axis=1, keepdims=True,
                             name="weighted_sum")

print(weighted_predictions)
print(weighted_sum)

Tensor("weighted_predictions:0", shape=(None, 3200, 2, 8, 1), dtype=float32)
Tensor("weighted_sum:0", shape=(None, 1, 2, 8, 1), dtype=float32)


Finalmente, se aplica la función squash para obtener las salidas de las cápsulas de la segunda capa al final de la primera iteración del algoritmo de enrutamiento por acuerdo $v_{j} = \frac{\left \| s_{j} \right \|^{2}}{1+\left \| s_{j} \right \|^{2}}\frac{s_{j}}{\left \| s_{j} \right \|}$

In [25]:
caps2_output_round_1 = squash(weighted_sum, axis=-2,
                              name="caps2_output_round_1")
caps2_output_round_1 # Se tienen que tener 8D vectores de salida para cada instancia (2)

<tf.Tensor 'caps2_output_round_1/mul:0' shape=(None, 1, 2, 8, 1) dtype=float32>

###### Ronda 2
Primero, se medirá como de cerca está cada vector predicho $\hat{u}_{j|i}$ de la actual vector de salida $v_{j}$ calculando su producto escalar  $\hat{u}_{j|i} \cdot  v_{j}$

In [26]:
caps2_predicted

<tf.Tensor 'caps2_predicted:0' shape=(None, 3200, 2, 8, 1) dtype=float32>

In [27]:
caps2_output_round_1

<tf.Tensor 'caps2_output_round_1/mul:0' shape=(None, 1, 2, 8, 1) dtype=float32>

In [28]:
caps2_output_round_1_tiled = tf.tile(
    caps2_output_round_1, [1, caps1_n_caps, 1, 1, 1],
    name="caps2_output_round_1_tiled")

In [29]:
agreement = tf.matmul(caps2_predicted, caps2_output_round_1_tiled,
                      transpose_a=True, name="agreement")

Ahora se pueden actualizar los pesos de enrutamiento $b_{i,j}$: 
$b_{i,j} \leftarrow b_{i,j} + \hat{u}_{j|i} \cdot  v_{j}$  (see Procedure 1, step 7, in the paper).
    

In [30]:
raw_weights_round_2 = tf.add(raw_weights, agreement,
                             name="raw_weights_round_2")

El resto de la ronda 2 es la misma que la ronda 1: 

In [31]:
routing_weights_round_2 = tf.nn.softmax(raw_weights_round_2,
                                        name="routing_weights_round_2")
weighted_predictions_round_2 = tf.multiply(routing_weights_round_2,
                                           caps2_predicted,
                                           name="weighted_predictions_round_2")
weighted_sum_round_2 = tf.reduce_sum(weighted_predictions_round_2,
                                     axis=1, keepdims=True,
                                     name="weighted_sum_round_2")
caps2_output_round_2 = squash(weighted_sum_round_2,
                              axis=-2,
                              name="caps2_output_round_2")

Se pueden repetir todas las rondas que se quiera, para ello habría que hacer exactamente los mismos pasos que en la ronda 2. Solo se harán para este TFM 2 rondas. 

In [32]:
caps2_output = caps2_output_round_2

#### Estimando las clases de probabilidad (longitud del vector)
Las longitudes de los vectores de salida representan las probabilidades de clase, para ello se puede usar tf.norm pero se corre el riesgo con los valores 0 como se explicó anteriormente, por ello se crea una función propia con epsilon:

In [33]:
def safe_norm(s, axis=-1, epsilon=1e-7, keep_dims=False, name=None):
    with tf.name_scope(name):
        squared_norm = tf.reduce_sum(tf.square(s), axis=axis,
                                     keepdims=keep_dims)
        return tf.sqrt(squared_norm + epsilon)

In [34]:
y_proba = safe_norm(caps2_output, axis=-2, name="y_proba")

Se predecirá la clase con mayor probabilidad estimada: 

In [35]:
y_proba_argmax = tf.argmax(y_proba, axis=2, name="y_proba")
y_proba_argmax

<tf.Tensor 'y_proba_1:0' shape=(None, 1, 1) dtype=int64>

In [36]:
y_pred = tf.squeeze(y_proba_argmax, axis=[1,2], name="y_pred") # squeeze elimina las dimensiones de tamaño 1 que ya no se necesitan
y_pred

<tf.Tensor 'y_pred:0' shape=(None,) dtype=int64>

#### Operaciones de entrenamiento

###### Etiquetas (labels)

In [37]:
y = tf.compat.v1.placeholder(shape=[None], dtype=tf.int64, name="y") # Placeholder para las etiquetas
y

<tf.Tensor 'y:0' shape=(None,) dtype=int64>

###### Margin loss

Se usa una función de perdida especial de margen que hace posible detectar dos o más tareas en cada conjunto de datos: 
$L_{k} = T_{k}max(0,m^{+} - \left \| v_{k} \right \|)^{2} + \lambda (1-T_{k})max(0,\left \| v_{k} \right \| -m^{-})^{2}$

Donde $T_{k}$ será 1 si la clase k está presente o 0 en otro caso. Los hiperparámetros $m^{+}$ y $m^{-}$ se instancian a 0.9 y 0.1. Y $\lambda$, que reduce la influencia de la perdida en las etiquetas que no pertecen a la clase correcta, se instancia a 0.5.

In [38]:
m_plus = 0.9
m_minus = 0.1
lambda_ = 0.5

In [39]:
T = tf.one_hot(y, depth=caps2_n_caps, name="T")

In [40]:
with tf.compat.v1.Session():
    print(T.eval(feed_dict={y: np.array([0, 1, 2, 3, 9])}))

[[1. 0.]
 [0. 1.]
 [0. 0.]
 [0. 0.]
 [0. 0.]]


Ahora, se calculará la norma de los vectores salida para cada salida de cápsula y cada instancia:

In [41]:
caps2_output

<tf.Tensor 'caps2_output_round_2/mul:0' shape=(None, 1, 2, 8, 1) dtype=float32>

In [42]:
caps2_output_norm = safe_norm(caps2_output, axis=-2, keep_dims=True,
                              name="caps2_output_norm")

Calculamos $max(0,m^{+} - \left \| v_{k} \right \|)^{2}$ y se cambia su dimension a (batch size, 2)

In [43]:
present_error_raw = tf.square(tf.maximum(0., m_plus - caps2_output_norm),
                              name="present_error_raw")
present_error = tf.reshape(present_error_raw, shape=(-1, 2),
                           name="present_error")

Calculamos $max(0,\left \| v_{k} \right \| -m^{-})^{2}$ y se cambia su dimension a (batch size, 2)

In [44]:
absent_error_raw = tf.square(tf.maximum(0., caps2_output_norm - m_minus),
                             name="absent_error_raw")
absent_error = tf.reshape(absent_error_raw, shape=(-1, 2),
                          name="absent_error")

Y ya calculamos el loss para cada instancia $L_{0}+L_{1}$ y calculamos la media. Lo que da un loss final: 

In [45]:
L = tf.add(T * present_error, lambda_ * (1.0 - T) * absent_error,
           name="L")

In [46]:
margin_loss = tf.reduce_mean(tf.reduce_sum(L, axis=1), name="margin_loss")

#### Reconstrucción 

Ahora es el momento de añadir la red decoder al final de la red capsular. Esta será una red de 3 capas completamente conectadas la cual aprenderá la reconstrucción de los datos de entrada basándose en la salida de la red capsular. Esto fuerza a la red capsular a mantener la información para reconstruir los datos. Además, regulariza el modelo, reduciendo el riesgo de overfitting y ayudando a generalizar.

###### Mask

Durante el entrenamiento, en lugar de enviar todas las salidas de las redes capsulares a la red decoder, solo se deben enviar los vectores de salida de las capsulas que correspondan a la tarea objetivo. Todas los otros vectores de salida deben ser enmascarados. Vamos, habrá que enmascarar todos menos el que más largo que será el que corresponderá a la entrada.

In [47]:
mask_with_labels = tf.compat.v1.placeholder_with_default(False, shape=(),
                                               name="mask_with_labels")

In [48]:
reconstruction_targets = tf.cond(mask_with_labels, # condition
                                 lambda: y,        # if True
                                 lambda: y_pred,   # if False
                                 name="reconstruction_targets")

In [49]:
reconstruction_mask = tf.one_hot(reconstruction_targets,
                                 depth=caps2_n_caps,
                                 name="reconstruction_mask")

In [50]:
reconstruction_mask

<tf.Tensor 'reconstruction_mask:0' shape=(None, 2) dtype=float32>

In [51]:
caps2_output

<tf.Tensor 'caps2_output_round_2/mul:0' shape=(None, 1, 2, 8, 1) dtype=float32>

In [52]:
reconstruction_mask_reshaped = tf.reshape(
    reconstruction_mask, [-1, 1, caps2_n_caps, 1, 1],
    name="reconstruction_mask_reshaped")

Aplicamos la máscara: 

In [53]:
caps2_output_masked = tf.multiply(
    caps2_output, reconstruction_mask_reshaped,
    name="caps2_output_masked")

In [54]:
caps2_output_masked

<tf.Tensor 'caps2_output_masked:0' shape=(None, 1, 2, 8, 1) dtype=float32>

In [55]:
 #  reshape operation to flatten the decoder's inputs
decoder_input = tf.reshape(caps2_output_masked,
                       [-1, caps2_n_caps * caps2_n_dims],
                       name="decoder_input")

###### Decoder
2 densas capas ReLu completamente conectadas seguidas de una capa de salida sigmoide

In [56]:
n_hidden1 = 512
n_hidden2 = 1024
n_output = 32 * 5 # 32 x 5

In [57]:
with tf.name_scope("decoder"):
    hidden1 = tf.keras.layers.Dense(n_hidden1, activation='relu', name="hidden1")(decoder_input) 
    hidden2 = tf.keras.layers.Dense(n_hidden2, activation='relu', name="hidden2")(hidden1)
    decoder_output = tf.keras.layers.Dense(n_output, activation='sigmoid', name="decoder_output")(hidden2)

###### Loss de la reconstrucción
Solo es la diferencia al cuadrado entre los datos de entrada y los datos reconstruidos

In [58]:
X_flat = tf.reshape(X, [-1, n_output], name="X_flat")
squared_difference = tf.square(X_flat - decoder_output,
                               name="squared_difference")
reconstruction_loss = tf.reduce_mean(squared_difference,
                                    name="reconstruction_loss")

###### Loss final
Es la suma del margin loss y del loss de la reconstrucción

In [59]:
alpha = 0.0005

loss = tf.add(margin_loss, alpha * reconstruction_loss, name="loss")

#### Entrenamiento y evaluación
Primero instanciamos el accuracy:

In [60]:
correct = tf.equal(y, y_pred, name="correct")
accuracy = tf.reduce_mean(tf.cast(correct, tf.float32), name="accuracy")

Los hiperparámetros de entrenamiento:

In [61]:
optimizer = tf.compat.v1.train.AdamOptimizer()
training_op = optimizer.minimize(loss, name="training_op")

Se crea la variable inicializadora y de guardado:

In [62]:
init = tf.compat.v1.global_variables_initializer()
saver = tf.compat.v1.train.Saver()

In [63]:
X_train[0:1].dtype

dtype('float32')

### 5 - Entrenamiento del modelo

In [64]:
n_epochs = 500
batch_size = 28
#restore_checkpoint = False

n_iterations_per_epoch = len(X_train) // batch_size
n_iterations_validation = len(X_dev) // batch_size
best_loss_val = np.infty
#checkpoint_path = "./my_capsule_network"

with tf.compat.v1.Session() as sess:
    #if restore_checkpoint and tf.compat.v1.train.checkpoint_exists(checkpoint_path):
    #    saver.restore(sess, checkpoint_path)
    #else:
    init.run()

    for epoch in range(n_epochs):
        b0 = 0
        c0 = 0
        for iteration in range(1, n_iterations_per_epoch + 1):
            X_batch, y_batch = X_train[b0:b0+batch_size], y_train[b0:b0+batch_size]
            # Run the training operation and measure the loss:
            print("X_batch, y_batch", np.shape(X_batch), np.shape(y_batch))
            _, loss_train = sess.run(
                [training_op, loss],
                feed_dict={X: X_batch.reshape([-1, 32, 5, 1]),
                           y: y_batch,
                           mask_with_labels: True})
            
            print("HOLAAAAAAAAAAAAAAaa2")
            print("\rIteration: {}/{} ({:.1f}%)  Loss: {:.5f}".format(
                      iteration, n_iterations_per_epoch,
                      iteration * 100 / n_iterations_per_epoch,
                      loss_train),
                  end="")
            b0+=batch_size

        # At the end of each epoch,
        # measure the validation loss and accuracy:
        loss_vals = []
        acc_vals = []
        
        for iteration in range(1, n_iterations_validation + 1):
            X_batch, y_batch = X_dev[c0:c0+batch_size], y_dev[c0:c0+batch_size]
            loss_val, acc_val = sess.run(
                    [loss, accuracy],
                    feed_dict={X: X_batch.reshape([-1, 32, 5, 1]),
                               y: y_batch})
            loss_vals.append(loss_val)
            acc_vals.append(acc_val)
            print("\rEvaluating the model: {}/{} ({:.1f}%) {}".format(
                      iteration, n_iterations_validation,
                      iteration * 100 / n_iterations_validation, c0),
                  end=" " * 10)
            c0+=batch_size
        loss_val = np.mean(loss_vals)
        acc_val = np.mean(acc_vals)
        print("\rEpoch: {}  Val accuracy: {:.4f}%  Loss: {:.6f}{}".format(
            epoch + 1, acc_val * 100, loss_val,
            " (improved)" if loss_val < best_loss_val else ""))

        # And save the model if it improved:
        if loss_val < best_loss_val:
            save_path = saver.save(sess, checkpoint_path)
            best_loss_val = loss_val

X_batch, y_batch (28, 32, 5, 1) (28,)


InvalidArgumentError: ignored